[konlpy tag URL]()

[korean stop words URL]()

[model 참고 URL](https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/)

In [ ]:
!pip install konlpy
!pip install pytorch-transformers

In [ ]:
import pandas as pd
import numpy as np
import json
import ast
from konlpy.tag import Okt, Hannanum, Komoran, Kkma
from sklearn.feature_extraction.text import TfidfVectorizer

import random
import copy
import time
import gc
import re
import torch

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os 
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import gc

import matplotlib.pyplot as plt

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
PATH="/content/drive/MyDrive/학교 프로젝트/2022 슈퍼챌린지 SW 해커톤"

In [ ]:
def make_category(x):
  if int(x[1])<=3:
    return int(x[1])-1
  elif int(x[1])==4:
    return -1
  else:
    return int(x[1])-2

with open(f"{PATH}/data/train/감성대화말뭉치(최종데이터)_Training.json",'r') as datafile:
    data=json.load(datafile)
train_data=pd.json_normalize(data)
train_data=pd.DataFrame(train_data)
data=""
with open(f"{PATH}/data/test/감성대화말뭉치(최종데이터)_Validation.json",'r') as datafile:
    data=json.load(datafile)
test_data=pd.json_normalize(data)
test_data=pd.DataFrame(test_data)

train_data["context"]=train_data['talk.content.HS01']+" "+train_data['talk.content.HS02']+" "+train_data['talk.content.HS03']
test_data["context"]=test_data['talk.content.HS01']+" "+test_data['talk.content.HS02']+" "+test_data['talk.content.HS03']
used_col=['profile.persona-id','profile.emotion.type', 'context']
for col in train_data.columns:
  if col not in used_col:
    train_data.drop(columns=col,inplace=True)
    test_data.drop(columns=col,inplace=True)

train_data.columns=["id","emotion","context"]
test_data.columns=["id","emotion","context"]

#train_data["context"]=train_data["context"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
#train_data['context'].replace('', np.nan, inplace=True)
#test_data["context"]=test_data["context"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
#test_data['context'].replace('', np.nan, inplace=True)
  
train_data["emotion"]=train_data["emotion"].apply(make_category)
train_data.drop(train_data[train_data["emotion"]==-1].index,inplace=True)
test_data["emotion"]=test_data["emotion"].apply(make_category)
test_data.drop(test_data[test_data["emotion"]==-1].index,inplace=True)

In [ ]:
class emotionDataset(Dataset):
    ''' context&emotion dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 2]
        label = self.df.iloc[idx, 1]
        return text, label

emotion_train_dataset = emotionDataset(train_data)
train_loader = DataLoader(emotion_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [ ]:
print(len(emotion_train_dataset),len(train_loader))

40827 20414


In [ ]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=5)
model.to(device)

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 4
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/1] Iteration 500 -> Train Loss: 1.7954, Accuracy: 0.177
[Epoch 1/1] Iteration 1000 -> Train Loss: 1.7913, Accuracy: 0.183
[Epoch 1/1] Iteration 1500 -> Train Loss: 1.7948, Accuracy: 0.167
[Epoch 1/1] Iteration 2000 -> Train Loss: 1.7873, Accuracy: 0.202
[Epoch 1/1] Iteration 2500 -> Train Loss: 1.7864, Accuracy: 0.197
[Epoch 1/1] Iteration 3000 -> Train Loss: 1.7798, Accuracy: 0.200
[Epoch 1/1] Iteration 3500 -> Train Loss: 1.7839, Accuracy: 0.192
[Epoch 1/1] Iteration 4000 -> Train Loss: 1.7708, Accuracy: 0.217
[Epoch 1/1] Iteration 4500 -> Train Loss: 1.7826, Accuracy: 0.205
[Epoch 1/1] Iteration 5000 -> Train Loss: 1.7462, Accuracy: 0.238
[Epoch 1/1] Iteration 5500 -> Train Loss: 1.6987, Accuracy: 0.278
[Epoch 1/1] Iteration 6000 -> Train Loss: 1.6769, Accuracy: 0.305
[Epoch 1/1] Iteration 6500 -> Train Loss: 1.6186, Accuracy: 0.326
[Epoch 1/1] Iteration 7000 -> Train Loss: 1.5620, Accuracy: 0.349
[Epoch 1/1] Iteration 7500 -> Train Loss: 1.5489, Accuracy: 0.378
[Epoch 1/1]

In [ ]:
# evaluation
model.eval()

emotion_eval_dataset = emotionDataset(test_data)
eval_loader = DataLoader(emotion_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test accuracy:  0.6438891058180398


In [ ]:
# evaluation
device="cpu"
model.to(device)
model.eval()
new_complaint="배고프다. 돈도 없고 춥고 배고파. 나는 거지야. 나는 아무짝에도 쓸모없는 사람인가봐."
encoded_list=[tokenizer.encode(new_complaint,add_special_tokens=True)]
padded_list=  [e + [0] * (512-len(e)) for e in encoded_list]
emotion_eval_dataset = emotionDataset(test_data)
eval_loader = DataLoader(emotion_eval_dataset, batch_size=2, shuffle=False, num_workers=2)
sample = torch.tensor(padded_list)
sample= sample.to(device)
label=torch.tensor([0])
label=label.to(device)
outputs = model(sample,labels=label)
_, logits = outputs

pred = torch.argmax(F.softmax(logits), dim=1)
print(pred)

tensor([0])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
new_complaint=[""]
seq=tokenizer.texts_to_sequences(new_complaint)
padded=pad_sequences(seq,maxlen=MAX_SEQUENCE_LENGTH)
pred=model.predict(padded)
print(np.argmax(pred))